In [2]:
%pip install kagglehub

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("saurabh00007/iriscsv")

print("Path to dataset files:", path)

c:\Users\manve\anaconda3\envs\pyspark_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 1.28k/1.28k [00:00<00:00, 612kB/s]

Extracting files...
Path to dataset files: C:\Users\manve\.cache\kagglehub\datasets\saurabh00007\iriscsv\versions\1


In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

In [3]:
spark = SparkSession.builder.appName("Iris Classification").getOrCreate()
spark

In [4]:
data = spark.read.csv("Iris.csv", inferSchema=True, header=True)

In [5]:
data.show()
data.count()

+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
|  6|          5.4|         3.9|          1.7|         0.4|Iris-setosa|
|  7|          4.6|         3.4|          1.4|         0.3|Iris-setosa|
|  8|          5.0|         3.4|          1.5|         0.2|Iris-setosa|
|  9|          4.4|         2.9|          1.4|         0.2|Iris-setosa|
| 10|          4.9|         3.1|          1.5|         0.1|Iris-setosa|
| 11|          5.4|         3.7|          1.5|         0.2|Iris-

150

In [6]:
data = data.na.drop()

In [7]:
data.count()

150

In [8]:
data = data.dropDuplicates()

In [9]:
data.count()

150

In [10]:
data.show()

+---+-------------+------------+-------------+------------+---------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|        Species|
+---+-------------+------------+-------------+------------+---------------+
| 73|          6.3|         2.5|          4.9|         1.5|Iris-versicolor|
|  2|          4.9|         3.0|          1.4|         0.2|    Iris-setosa|
| 31|          4.8|         3.1|          1.6|         0.2|    Iris-setosa|
| 16|          5.7|         4.4|          1.5|         0.4|    Iris-setosa|
| 99|          5.1|         2.5|          3.0|         1.1|Iris-versicolor|
|117|          6.5|         3.0|          5.5|         1.8| Iris-virginica|
|124|          6.3|         2.7|          4.9|         1.8| Iris-virginica|
|144|          6.8|         3.2|          5.9|         2.3| Iris-virginica|
|148|          6.5|         3.0|          5.2|         2.0| Iris-virginica|
|119|          7.7|         2.6|          6.9|         2.3| Iris-virginica|
| 36|       

In [11]:
# Preprocess data
feature_columns = ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(data)

In [12]:
# Rename target column to "label" for Spark MLlib
data = data.withColumnRenamed("species", "label")

In [13]:
data.show()

+---+-------------+------------+-------------+------------+---------------+-----------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|          label|         features|
+---+-------------+------------+-------------+------------+---------------+-----------------+
| 73|          6.3|         2.5|          4.9|         1.5|Iris-versicolor|[6.3,2.5,4.9,1.5]|
|  2|          4.9|         3.0|          1.4|         0.2|    Iris-setosa|[4.9,3.0,1.4,0.2]|
| 31|          4.8|         3.1|          1.6|         0.2|    Iris-setosa|[4.8,3.1,1.6,0.2]|
| 16|          5.7|         4.4|          1.5|         0.4|    Iris-setosa|[5.7,4.4,1.5,0.4]|
| 99|          5.1|         2.5|          3.0|         1.1|Iris-versicolor|[5.1,2.5,3.0,1.1]|
|117|          6.5|         3.0|          5.5|         1.8| Iris-virginica|[6.5,3.0,5.5,1.8]|
|124|          6.3|         2.7|          4.9|         1.8| Iris-virginica|[6.3,2.7,4.9,1.8]|
|144|          6.8|         3.2|          5.9|         2.3| 

In [14]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="label", outputCol="label-indexed")
data = indexer.fit(data).transform(data)
data.show()

+---+-------------+------------+-------------+------------+---------------+-----------------+-------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|          label|         features|label-indexed|
+---+-------------+------------+-------------+------------+---------------+-----------------+-------------+
| 73|          6.3|         2.5|          4.9|         1.5|Iris-versicolor|[6.3,2.5,4.9,1.5]|          1.0|
|  2|          4.9|         3.0|          1.4|         0.2|    Iris-setosa|[4.9,3.0,1.4,0.2]|          0.0|
| 31|          4.8|         3.1|          1.6|         0.2|    Iris-setosa|[4.8,3.1,1.6,0.2]|          0.0|
| 16|          5.7|         4.4|          1.5|         0.4|    Iris-setosa|[5.7,4.4,1.5,0.4]|          0.0|
| 99|          5.1|         2.5|          3.0|         1.1|Iris-versicolor|[5.1,2.5,3.0,1.1]|          1.0|
|117|          6.5|         3.0|          5.5|         1.8| Iris-virginica|[6.5,3.0,5.5,1.8]|          2.0|
|124|          6.3|         

In [15]:
train, test = data.randomSplit([0.7, 0.3])

In [16]:
lr = LogisticRegression(featuresCol="features", labelCol="label-indexed")
model = lr.fit(train)

In [17]:
predictions = model.transform(test)
predictions.select("features", "label", "prediction").show()

+-----------------+---------------+----------+
|         features|          label|prediction|
+-----------------+---------------+----------+
|[4.6,3.4,1.4,0.3]|    Iris-setosa|       0.0|
|[4.9,3.1,1.5,0.1]|    Iris-setosa|       0.0|
|[5.4,3.7,1.5,0.2]|    Iris-setosa|       0.0|
|[4.8,3.4,1.6,0.2]|    Iris-setosa|       0.0|
|[5.7,3.8,1.7,0.3]|    Iris-setosa|       0.0|
|[4.6,3.6,1.0,0.2]|    Iris-setosa|       0.0|
|[5.1,3.3,1.7,0.5]|    Iris-setosa|       0.0|
|[5.2,3.4,1.4,0.2]|    Iris-setosa|       0.0|
|[4.8,3.1,1.6,0.2]|    Iris-setosa|       0.0|
|[5.5,4.2,1.4,0.2]|    Iris-setosa|       0.0|
|[5.5,3.5,1.3,0.2]|    Iris-setosa|       0.0|
|[4.9,3.1,1.5,0.1]|    Iris-setosa|       0.0|
|[5.1,3.4,1.5,0.2]|    Iris-setosa|       0.0|
|[4.4,3.2,1.3,0.2]|    Iris-setosa|       0.0|
|[5.0,3.5,1.6,0.6]|    Iris-setosa|       0.0|
|[5.0,3.3,1.4,0.2]|    Iris-setosa|       0.0|
|[6.4,3.2,4.5,1.5]|Iris-versicolor|       1.0|
|[6.9,3.1,4.9,1.5]|Iris-versicolor|       1.0|
|[5.9,3.0,4.2

In [18]:
data.columns

['Id',
 'SepalLengthCm',
 'SepalWidthCm',
 'PetalLengthCm',
 'PetalWidthCm',
 'label',
 'features',
 'label-indexed']

In [ ]:
data.printSchema()


In [21]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

# Create user data
user_data = [Row(SepalLengthCm=6.5, SepalWidthCm=3.0, PetalLengthCm=5.2, PetalWidthCm=2.0)]
user_df = spark.createDataFrame(user_data)

# Transform user data to include features
user_df = assembler.transform(user_df)

# Predict user data
user_predictions = model.transform(user_df)

# Show predictions
user_predictions.select("features", "prediction").show()


Py4JJavaError: An error occurred while calling o326.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 128.0 failed 1 times, most recent failure: Lost task 0.0 in stage 128.0 (TID 70) (Anvesh executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(R

In [20]:
data.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- label: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- label-indexed: double (nullable = false)

